# Training file (testing if training works)

In [1]:
import sys
sys.path.append('../')

In [2]:
import torch
from torch.utils.data import DataLoader
from torchvision import transforms
from torch.nn import CTCLoss
import torch.optim as optim

from src.training.trainer import train_model
from src.dataset.custom_dataset import OdometerDataset
from src.dataset.base_dataset import base_collate_fn
from src.models.crnn import CRNN

/home/yannou/OneDrive/Documents/3_PRO/Box2Read/env-ocr/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda', index=0)

In [4]:

data_dir = '/home/yannou/OneDrive/Documents/3_PRO/carviz/data/ocr/odometer_reader/milestone_box'
labels_file = '/home/yannou/OneDrive/Documents/3_PRO/carviz/data/ocr/odometer_reader/milestone_box/milestone_labels.json'

# Définir les transformations
transform = transforms.Compose([
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
    transforms.Grayscale(num_output_channels=1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

transform_val = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

# Initialiser le dataset
dataset = OdometerDataset(root_dir=data_dir, split="train", labels_file=labels_file, img_height=32, img_width=100, transform=transform)
dataset_val = OdometerDataset(root_dir=data_dir, split="val", labels_file=labels_file, img_height=32, img_width=100, transform=transform_val)


# Créer les DataLoaders
train_loader = DataLoader(dataset, batch_size=64, shuffle=True, collate_fn=base_collate_fn)
valid_loader = DataLoader(dataset_val, batch_size=64, shuffle=False, collate_fn=base_collate_fn)



In [5]:
# Initialiser le modèle
num_class = len(OdometerDataset.LABEL2CHAR) + 1
crnn = CRNN(img_channel=1, img_height=32, img_width=100, num_class=num_class, model_size="n", leaky_relu=True).to(device)

# Définir les paramètres d'entraînement
lr = 0.001
epochs = 150
decode_method = 'beam_search'
beam_size = 10
label2char = OdometerDataset.LABEL2CHAR

In [ ]:
print(f"Working on {device}")
# Appeler la fonction train_model
trained_model = train_model(
    model=crnn,
    train_loader=train_loader,
    valid_loader=valid_loader,
    label2char=label2char,
    device=device,
    lr=lr,
    epochs=epochs,
    decode_method=decode_method,
    beam_size=beam_size,
    criterion=CTCLoss(reduction='sum', zero_infinity=True).to(device),
    optimizer=optim.Adam(crnn.parameters(), lr=lr),
    project_name="odometer-reader",
    run_name="crnn-n"
)



Working on cuda:0


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: yann-t (carviz-com). Use `wandb login --relogin` to force relogin


Epochs:   1%|          | 1/150 [01:24<3:29:19, 84.29s/it]

Epoch 1: train_loss=20.479480619789534, train_accuracy=0.0, val_loss=13.71358294033165, val_accuracy=0.0


Epochs:   1%|▏         | 2/150 [02:44<3:22:19, 82.02s/it]

...

Epoch 50: train_loss=1.8740412360907772, train_accuracy=0.5980360065466448, val_loss=1.2900974567119892, val_accuracy=0.7079530638852672


Epochs:  34%|███▍      | 51/150 [59:09<1:50:09, 66.76s/it]

Epoch 51: train_loss=1.8698155779300072, train_accuracy=0.6062193126022913, val_loss=1.2580486929245691, val_accuracy=0.7105606258148631


Epochs:  35%|███▍      | 52/150 [1:00:18<1:50:23, 67.59s/it]

Epoch 52: train_loss=1.8284869229930123, train_accuracy=0.6062193126022913, val_loss=1.3615165421984496, val_accuracy=0.6962190352020861
